## [Analyze and Debug LlamaIndex Applications with PostHog and Langfuse ](https://langfuse.com/guides/cookbook/integration_llama_index_posthog_mistral)

Build a RAG application with LlamaIndex, observe the steps with Langfuse, and analyze the data in PostHog.

What is LlamaIndex?

LlamaIndex (GitHub) is a data framework designed to connect LLMs with external data sources. It helps structure, index, and query data effectively. This makes it easier for developers to build advanced LLM applications.

### Build a Simple RAG Application with LlamaIndex and Mistral

Create a chat application that provides answers to questions about hedgehog care. LlamaIndex is used to vectorize a hedgehog care guide with the Mistral 8x22B model. All model generations are then traced using Langfuse’s LlamaIndex integration.

Finally, the [PostHog integration](https://langfuse.com/integrations/analytics/posthog) allows you to view detailed analytics about your hedgehog application directly in PostHog.

#### Step 1: Set Up LlamaIndex and Mistral

First, we set our Mistral API key as an environment variable. If you haven’t already, [sign up for a Mistral account](https://console.mistral.ai/). Then subscribe to a free trial or billing plan, after which you’ll be able to generate an API key (💡 You can use any other model supported by LlamaIndex; we just use Mistral in this cookbook).

Then, we use LlamaIndex to initialize both a Mistral language model and an embedding model. We then set these models in the LlamaIndex Settings object:

In [1]:
%pip install llama-index llama-index-llms-mistralai llama-index-embeddings-mistralai nest_asyncio --upgrade

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     --------- ------------------------------ 1.8/7.6 MB 10.1 MB/s eta 0:00:01
     ------------- -------------------------- 2.6/7.6 MB 8.4 MB/s eta 0:00:01
     -------------------- ------------------- 3.9/7.6 MB 6.7 MB/s eta 0:00:01
     ----------------------- ---------------- 4.5/7.6 MB 5.3 MB/s eta 0:00:01
     ---------------------------------- ----- 6.6/7.6 MB 6.3 MB/s eta 0:00:01
     ------------------------------------- -- 7.1/7.6 MB 6.3 MB/s eta 0:00:01
     ------------------------------------- -- 7.1/7.6 MB 6.3 MB/s eta 0:00:01
     ---------------------------------------- 7.6/7.6 MB 4.9 MB/s  0:00:01

  Attempting uninstall: llama-index-core

    Found existing installation: llama-index-core 0.13.5

    Uninstalling llama-index-core-0.13.5:

   ---------------------------------------- 0/4 [llama-index-core]
      Successfully uninstalled llam

In [1]:
# Set the Mistral API key
import os
 
os.environ["MISTRAL_API_KEY"] = "ChJxeEAnblw7oKELK0cJMVORCMXgrpYn"
 
# Ensures that sync and async code can be used together without issues
import nest_asyncio
 
nest_asyncio.apply()
 
# Import and set up llama index
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.core import Settings
 
# Define your LLM and embedding model
llm = MistralAI(model="open-mixtral-8x22b", temperature=0.1)
embed_model = MistralAIEmbedding(model_name="mistral-embed")
 
# Set the LLM and embedding model in the Settings object
Settings.llm = llm
Settings.embed_model = embed_model

#### Step 2: Initialize Langfuse

In [5]:
%pip install langfuse openinference-instrumentation-llama-index wget

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9712 sha256=963763a49bbe535daf69c10fd622f11f9c3b4da9c643419b1e22520dd92c7e1d
  Stored in directory: c:\users\slsxz\appdata\local\pip\cache\wheels\66\9e\4d\df43e9e196c24d1dcbbc9b362e738a39d63cd4bbfde8745380
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'wget' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wget'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [2]:
import os

import os
 
# Get keys for your project from the project settings page: https://cloud.langfuse.com
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-b26f6b38-1d21-4efb-b0ed-5057200b2148" 
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-842c4b9d-4e56-4325-9df5-5a09408f42b1" 

os.environ["LANGFUSE_HOST"] = "http://localhost:3000"

# os.environ["OPENAI_API_KEY"] = "73c80b33ad68446ea3f059efe5c1a65f.T2PZjYiHcT2JYx2a"
# os.environ["OPENAI_BASE_URL"] = "https://open.bigmodel.cn/api/paas/v4"


In [3]:
from langfuse import get_client
 
langfuse = get_client()
 
# Verify connection
if langfuse.auth_check():
    print("Langfuse client is authenticated and ready!")
else:
    print("Authentication failed. Please check your credentials and host.")
 

Langfuse client is authenticated and ready!


Initialize the [OpenInference LlamaIndex instrumentation](https://arize.com/docs/phoenix/integrations/frameworks/llamaindex). This third-party instrumentation **automatically captures LlamaIndex operations and exports OpenTelemetry (OTel) spans to Langfuse.**

In [4]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
 
# Initialize LlamaIndex instrumentation
LlamaIndexInstrumentor().instrument()

#### Step 3: Download data

We download the file we want to use for RAG. In this example, we use a hedgehog care guide pdf file to enable the language model to answer questions about caring for hedgehogs 🦔.

In [12]:
import wget
 
# url = "https://www.pro-igel.de/downloads/merkblaetter_engl/wildtier_engl.pdf"
# url = "https://earps.org/media/2022/02/Hedgehog-Care-Guide.pdf"
# wget.download(url, "./hedgehog.pdf")   # saves as ./hedgehog.pdf

Load the pdf using the LlamaIndex [SimpleDirectoryReader](https://docs.llamaindex.ai/en/stable/module_guides/loading/simpledirectoryreader/).

In [5]:
from llama_index.core import SimpleDirectoryReader
 
hedgehog_docs = SimpleDirectoryReader(
    input_files=["./hedgehog.pdf"]
).load_data()

#### Step 4: Build RAG on the hedgehog doc

create vector embeddings of the hedgehog document using [VectorStoreIndex](https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/) and then convert it into a [queryable engine](https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/) to retrieve information based on queries.

In [6]:
from llama_index.core import VectorStoreIndex
 
hedgehog_index = VectorStoreIndex.from_documents(hedgehog_docs)
hedgehog_query_engine = hedgehog_index.as_query_engine(similarity_top_k=5)

Finally, to put everything together, we query the engine and print a response:

In [7]:
response = hedgehog_query_engine.query("Which hedgehogs require help?")
print(response)

Hedgehogs that are affected by Wobbly Hedgehog Syndrome (WHS) require help. As the condition progresses, they may need assistance with feeding and hydration through syringe feeding, as well as support for mobility and independence. Additionally, hedgehogs that have been injured or are experiencing other health issues may require help, including immediate veterinary treatment.


In [ ]:
# Summarize cost by model
import pandas as pd

trace = langfuse.api.trace.get("e5cf526e8c6b302c496cf9305e697916")
observations = trace.observations


def summarize_usage(observations):
    """Summarize usage data grouped by model."""
    usage_data = []

    for obs in observations:
        usage = obs.usage
        if usage:
            usage_data.append(
                {
                    "model": obs.model,
                    "input_tokens": usage.input,
                    "output_tokens": usage.output,
                    "total_tokens": usage.total,
                }
            )

    df = pd.DataFrame(usage_data)
    if df.empty:
        return pd.DataFrame()

    summary = df.groupby("model").sum()
    return summary


# Example usage (assuming `observations` is defined as in the provided code):
summary_df = summarize_usage(observations)
summary_df

,input_tokens,output_tokens,total_tokens
model,,,
open-mixtral-8x22b,3263,98,3361


#### Step 5: (Optional) Implement user feedback to see how your application is performing

To monitor the quality of your hedgehog chat application, you can use Langfuse Scores to store user feedback (e.g. thumps up/down or comments). These scores can then be analyzed in PostHog.

Scores are used to evaluate single observations or entire traces. You can create them via the annotation workflow in the Langfuse UI, run model-based evaluation or ingest via the SDK as we do it in this example.

To get the context of the current observation, we use the observe() decorator and apply it to the hedgehog_helper() function.

In [17]:
from langfuse import observe, get_client

langfuse = get_client()

# Langfuse observe() decorator to automatically create a trace for the top-level function and spans for any nested functions.
@observe()
def hedgehog_helper(user_message):
    response = hedgehog_query_engine.query(user_message)
    trace_id = langfuse.get_current_trace_id()
 
    print(response)
 
    return trace_id

trace_id = hedgehog_helper("Can I keep the hedgehog as a pet?")

# Score the trace
langfuse.create_score(
    trace_id=trace_id,
    name="user-explicit-feedback",
    value=0.9,
    data_type="NUMERIC",  # optional, inferred if not provided
    comment="Good to know!",  # optional
)


Before deciding to keep a hedgehog as a pet, it's important to consider several factors. Firstly, ensure you have enough time and attention for a hedgehog and a suitable place to keep its cage away from other pets. Remember, hedgehogs are nocturnal animals and their schedule cannot be changed.

You should also be prepared to handle the hedgehog's quills, which are sharper than they appear. It's advisable to visit a reputable breeder or someone who owns hedgehogs to see what the quills are like.

Hedgehogs require a constant temperature of 73-78°F (23-25°C) in their cage to prevent them from attempting hibernation, which can be dangerous. This means you'll need a space heater, ceramic heat emitter, or other type of heating setup.

It's also crucial to understand that hedgehogs are solitary animals and should not be housed together. Males should never be housed together as they are territorial and almost never get along. Female-female pairing can work under the right circumstances, but i

#### Step 6: See your data in PostHog

[PostHog](https://us.posthog.com/)